In [9]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import joblib  # For saving and loading models
import pandas as pd
import numpy as np

# Load and preprocess the dataset
df = pd.read_csv(r"C:\Users\abhin\Desktop\Nalco\backend\2_dec_final_short.csv")

# Define features and target
X = df[['UTS', 'Elongation', 'Conductivity']]
y = df[['SI', 'FE', 'MetalTemp', 'CastingWheel_RPM', 'CoolingWaterPressure',
        'CoolingWaterTemp', 'CastBarEntryTemp', 'RollingMill_RPM', 'EmulsionTemp',
         'EmulsionPressure', 'RodQuenchWaterPressure']]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the base Random Forest model
rf_base = MultiOutputRegressor(RandomForestRegressor(random_state=42))

# Hyperparameter tuning using GridSearchCV
param_grid = {
    "estimator__n_estimators": [100, 200, 300],
    "estimator__max_depth": [10, 20, None],
    "estimator__min_samples_split": [2, 5],
    "estimator__min_samples_leaf": [1, 2],
    "estimator__max_features": ['sqrt', None]
}

grid_search = GridSearchCV(
    estimator=rf_base,
    param_grid=param_grid,
    cv=5,
    scoring='r2',
    n_jobs=-1,
    verbose=2
)

# Fit the GridSearchCV
grid_search.fit(X_train, y_train)

# Best parameters and best model
best_rf = grid_search.best_estimator_
print("Best parameters from GridSearchCV:", grid_search.best_params_)

# Evaluate the optimized model
y_pred = best_rf.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("\nOptimized Random Forest Performance:")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"R^2 Score: {r2:.4f}")

# Save the scaler and trained model as .pkl files
joblib.dump(scaler, 'rev_random_forest_scaler.pkl')
joblib.dump(best_rf, 'rev_optimized_random_forest_model.pkl')
print("\nScaler and model have been saved as 'rev_random_forest_scaler.pkl' and 'rev_optimized_random_forest_model.pkl'.")

Fitting 5 folds for each of 72 candidates, totalling 360 fits
Best parameters from GridSearchCV: {'estimator__max_depth': 10, 'estimator__max_features': 'sqrt', 'estimator__min_samples_leaf': 2, 'estimator__min_samples_split': 5, 'estimator__n_estimators': 300}

Optimized Random Forest Performance:
Mean Squared Error (MSE): 838.8786
Root Mean Squared Error (RMSE): 28.9634
R^2 Score: 0.1451

Scaler and model have been saved as 'random_forest_scaler.pkl' and 'optimized_random_forest_model.pkl'.


In [12]:
# Load the saved model and scaler
def load_model_and_scaler():
    loaded_scaler = joblib.load(r'C:\Users\abhin\Desktop\Nalco\backend\rev_random_forest_scaler.pkl')
    loaded_model = joblib.load(r'C:\Users\abhin\Desktop\Nalco\backend\rev_optimized_random_forest_model.pkl')
    return loaded_scaler, loaded_model

# Accept runtime input and make predictions
def get_runtime_input():
    print("\nEnter the input parameters for prediction:")

    # Define input ranges for each parameter
    while True:
        try:
            UTS = float(input("UTS [9 - 16]: "))
            if 9 <= UTS <= 16:
                break
            else:
                print("Invalid input! Please enter a value between 9 - 1.6")
        except ValueError:
            print("Invalid input! Please enter a numeric value.")

    while True:
        try:
            Elongation = float(input("Elongation [4 - 22]: "))
            if 4 <= Elongation <= 22:
                break
            else:
                print("Invalid input! Please enter a value between 4 - 22")
        except ValueError:
            print("Invalid input! Please enter a numeric value.")

    while True:
        try:
            Conductivity = float(input("Conductivity [59.5 - 62]: "))
            if 59.5 <= Conductivity <= 62:
                break
            else:
                print("Invalid input! Please enter a value between 59.5 - 62.")
        except ValueError:
            print("Invalid input! Please enter a numeric value.")

    # Return the inputs as a DataFrame
    user_input = pd.DataFrame({
        'UTS': [UTS],
        'Elongation': [Elongation],
        'Conductivity': [Conductivity],
    })

    return user_input

# Function to predict properties based on user input
def make_prediction_from_user_input():
    # Load scaler and model
    scaler, model = load_model_and_scaler()
    
    # Get user input
    user_input = get_runtime_input()
    
    # Scale the user input
    user_input_scaled = scaler.transform(user_input)
    
    # Make predictions
    predictions = model.predict(user_input_scaled)
    
    # Display the predictions
    print("\nPredicted Properties:")
    print(f"SI: {predictions[0][0]:.4f}")
    print(f"FE: {predictions[0][1]:.4f}")
    print(f"MetalTemp: {predictions[0][2]:.4f}")
    print(f"CastingWheel_RPM: {predictions[0][3]:.4f}")
    print(f"CoolingWaterPressure: {predictions[0][4]:.4f}")
    print(f"CoolingWaterTemp: {predictions[0][5]:.4f}")
    print(f"CastBarEntryTemp: {predictions[0][6]:.4f}")
    print(f"RollingMill_RPM: {predictions[0][7]:.4f}")
    print(f"EmulsionTemp: {predictions[0][8]:.4f}")
    print(f"EmulsionPressure: {predictions[0][9]:.4f}")
    # print(f"RodQuenchWaterPressure: {predictions[0][10]:.4f}")

# Main loop to allow repeated predictions
while True:
    print("\n--- Aluminum Wire Rod Property Prediction ---")
    make_prediction_from_user_input()
    cont = input("\nDo you want to make another prediction? (yes/no): ").strip().lower()
    if cont != 'yes':
        print("Exiting prediction tool. Goodbye!")
        break

# Optimized Random Forest Performance:
# Mean Squared Error (MSE): 795.4086
# Root Mean Squared Error (RMSE): 28.2030
# R^2 Score: 0.1032


--- Aluminum Wire Rod Property Prediction ---

Enter the input parameters for prediction:

Predicted Properties:
SI: 0.0751
FE: 0.1930
MetalTemp: 716.8120
CastingWheel_RPM: 1.9469
CoolingWaterPressure: 4.2049
CoolingWaterTemp: 33.4944
CastBarEntryTemp: 559.4499
RollingMill_RPM: 815.6391
EmulsionTemp: 59.7890
EmulsionPressure: 2.2990
Exiting prediction tool. Goodbye!
